## Kaggle – DataTops®
Luismi ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a Luismi a mejorar ese modelo?

## Métrica: 
Error de raíz cuadrada media (RMSE) es la desviación estándar de los valores residuales (errores de predicción). Los valores residuales son una medida de la distancia de los puntos de datos de la línea de regresión; RMSE es una medida de cuál es el nivel de dispersión de estos valores residuales. En otras palabras, le indica el nivel de concentración de los datos en la línea de mejor ajuste.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## Librerías

In [23]:
import numpy as np
import pandas as pd
import toolbox_ML as tb


from PIL import Image
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
import urllib.request

## Datos

In [24]:
# Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("./data/train.csv", index_col = 0)
df.index.name = None

In [25]:
df_bck_up = df.copy()

## Exploración de los datos

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           912 non-null    object 
 1   Product           912 non-null    object 
 2   TypeName          912 non-null    object 
 3   Inches            912 non-null    float64
 4   ScreenResolution  912 non-null    object 
 5   Cpu               912 non-null    object 
 6   Ram               912 non-null    object 
 7   Memory            912 non-null    object 
 8   Gpu               912 non-null    object 
 9   OpSys             912 non-null    object 
 10  Weight            912 non-null    object 
 11  Price_in_euros    912 non-null    float64
dtypes: float64(2), object(10)
memory usage: 92.6+ KB


In [27]:
df.isna().sum()

Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Memory              0
Gpu                 0
OpSys               0
Weight              0
Price_in_euros      0
dtype: int64

In [28]:
df.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


In [29]:
df.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00
229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42kg,2456.34


In [30]:
tb.describe_df(df)

,DATA_TYPE,MISSINGS (%),UNIQUE_VALUES,CARDIN (%)
Company,object,0,19,2.08
Product,object,0,480,52.63
TypeName,object,0,6,0.66
Inches,float64,0,17,1.86
ScreenResolution,object,0,36,3.95
Cpu,object,0,107,11.73
Ram,object,0,9,0.99
Memory,object,0,37,4.06
Gpu,object,0,93,10.2
OpSys,object,0,9,0.99


In [31]:
tb.tipifica_variables(df)

,variable,tipo
0,Company,Numérica Discreta
1,Product,Numérica Continua
2,TypeName,Categórica
3,Inches,Numérica Discreta
4,ScreenResolution,Numérica Discreta
5,Cpu,Numérica Discreta
6,Ram,Categórica
7,Memory,Numérica Discreta
8,Gpu,Numérica Discreta
9,OpSys,Categórica


## Procesado de datos

In [32]:
df[df.duplicated()].sort_values("Product")

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
1305,HP,15-AC110nv (i7-6500U/6GB/1TB/Radeon,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,764.00
1295,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg,289.00
1318,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1304,Lenovo,IdeaPad 100S-14IBR,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,229.00
1283,Lenovo,IdeaPad Y700-15ISK,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8GB,1TB HDD,Nvidia GeForce GTX 960M,Windows 10,2.6kg,899.00
1311,Lenovo,IdeaPad Y700-15ISK,Notebook,15.6,IPS Panel Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,8GB,1TB HDD,Nvidia GeForce GTX 960M,Windows 10,2.6kg,899.00
1310,Dell,Inspiron 3552,Notebook,15.6,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,500GB HDD,Intel HD Graphics,Windows 10,2.20kg,379.00
1285,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R5 M430,Linux,2.3kg,805.99
1294,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i3 7100U 2.4GHz,6GB,1TB HDD,Intel HD Graphics 620,Windows 10,2.3kg,459.00
1313,Dell,Inspiron 3567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R5 M430,Linux,2.3kg,805.99


In [33]:
# Viendo esos duplicados uno a uno, procedemos a eliminar los que realmente están duplicados utilizando su índice:
df.drop(index = [1313, 1318, 1320, 1315, 1316, 1303], inplace = True)

In [34]:
df["inches"] = df.Inches.astype(float)
df['ram_gb'] = df['Ram'].str.replace('GB', '').astype(int)
df["weight_kg"] = df['Weight'].str.replace('kg', '').astype(float)
df.drop(columns = ["Inches", "Ram", "Weight"], inplace = True)

In [35]:
# Ordenamos las features categóricas de menor precio de median a mayor precio para poder convertirlas en numéricas:
# categories_company = df.groupby(df["Company"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_typename = df.groupby(df["TypeName"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_resolution = df.groupby(df["ScreenResolution"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_cpu = df.groupby(df["Cpu"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_memory = df.groupby(df["Memory"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_gpu = df.groupby(df["Gpu"])["Price_in_euros"].median().sort_values().index.to_list()

# total_cat = [categories_company, categories_resolution, categories_cpu, categories_memory, categories_gpu]
# cols = ["Company", "ScreenResolution", "Cpu", "Memory", "Gpu"]

# enc = OrdinalEncoder(categories = total_cat, dtype= int, handle_unknown = "use_encoded_value", unknown_value = -1)
# df[cols] = enc.fit_transform(df[cols])
# df.head()

In [36]:
feat_sel = tb.get_features_cat_regression(df, "Price_in_euros")
print(feat_sel)

['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Memory', 'Gpu', 'OpSys']


In [37]:
tb.get_features_num_regression(df, "Price_in_euros", 0.6)

[]

In [38]:
df = pd.get_dummies(df, columns = feat_sel, dtype = int)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 906 entries, 755 to 229
Columns: 791 entries, Price_in_euros to OpSys_macOS
dtypes: float64(3), int32(788)
memory usage: 2.8 MB


Nuestro target es la columna `Price_in_euros`

In [40]:
target = "Price_in_euros"

-----------------------------------------------------------------------------------------------------------------

## Modelado

### 1. Definir X e y

In [42]:
X = df.drop(columns = [target])
y = df[target]

### 2. Dividir X_train, X_test, y_train, y_test

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [44]:
X_train.head()

,inches,ram_gb,weight_kg,Company_Acer,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,Company_HP,Company_Huawei,Company_LG,Company_Lenovo,Company_MSI,Company_Mediacom,Company_Microsoft,Company_Razer,Company_Samsung,Company_Toshiba,Company_Vero,Company_Xiaomi,Product_110-15ACL (A6-7310/4GB/500GB/W10),Product_14-am079na (N3710/8GB/2TB/W10),Product_15-AC110nv (i7-6500U/6GB/1TB/Radeon,Product_15-AY023na (N3710/8GB/2TB/W10),Product_15-BS026nv (i5-7200U/8GB/256GB/Radeon,Product_15-BS078nr (i7-7500U/8GB/1TB/W10),Product_15-BS101nv (i7-8550U/8GB/256GB/FHD/W10),Product_15-BW004nv (A9-9420/4GB/256GB/Radeon,Product_15-BW037na (A9-9420/4GB/1TB/Radeon,Product_15-BW091ND (A9-9420/6GB/1TB,Product_15-BW094nd (A6-9220/8GB/128GB/W10),Product_15-ay047nv (i3-6006U/6GB/1TB/Radeon,Product_15-bs002nv (i3-6006U/4GB/128GB/FHD/W10),Product_15-bs005nv (i3-6006U/4GB/1TB,Product_15-bs012nv (i7-7500U/8GB/1TB/Radeon,Product_15-bs025nv (i5-7200U/8GB/256GB/W10),Product_15-bs053od (i7-7500U/6GB/1TB/W10),Product_15-bs078cl (i7-7500U/8GB/2TB/W10),...,Gpu_Nvidia GeForce 940MX,Gpu_Nvidia GeForce GT 940MX,Gpu_Nvidia GeForce GTX 1050,Gpu_Nvidia GeForce GTX 1050 Ti,Gpu_Nvidia GeForce GTX 1050M,Gpu_Nvidia GeForce GTX 1050Ti,Gpu_Nvidia GeForce GTX 1060,Gpu_Nvidia GeForce GTX 1070,Gpu_Nvidia GeForce GTX 1070M,Gpu_Nvidia GeForce GTX 1080,Gpu_Nvidia GeForce GTX 940M,Gpu_Nvidia GeForce GTX 940MX,Gpu_Nvidia GeForce GTX 950M,Gpu_Nvidia GeForce GTX 960,Gpu_Nvidia GeForce GTX 960<U+039C>,Gpu_Nvidia GeForce GTX 960M,Gpu_Nvidia GeForce GTX 965M,Gpu_Nvidia GeForce GTX 970M,Gpu_Nvidia GeForce GTX 980,Gpu_Nvidia GeForce GTX 980M,Gpu_Nvidia GeForce GTX1050 Ti,Gpu_Nvidia GeForce MX130,Gpu_Nvidia GeForce MX150,Gpu_Nvidia Quadro M1000M,Gpu_Nvidia Quadro M1200,Gpu_Nvidia Quadro M2000M,Gpu_Nvidia Quadro M2200M,Gpu_Nvidia Quadro M3000M,Gpu_Nvidia Quadro M500M,Gpu_Nvidia Quadro M620,Gpu_Nvidia Quadro M620M,OpSys_Android,OpSys_Chrome OS,OpSys_Linux,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS
153,15.6,16,2.56,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
275,13.3,8,1.37,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
111,15.6,16,2.20,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
207,15.6,16,3.20,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
771,12.3,8,1.10,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [45]:
y_train.head()

153    1249.26
275    1958.90
111    1299.00
207    1749.00
771    1559.00
Name: Price_in_euros, dtype: float64

### 3. Baseline de modelos


In [48]:
cat_reg = CatBoostRegressor(n_estimators = 100, depth = 5, learning_rate = 0.5, verbose = 100)

cat_reg.fit(X_train, y_train)

0:	learn: 526.5641273	total: 4.3ms	remaining: 426ms
99:	learn: 127.4665295	total: 312ms	remaining: 0us


In [ ]:
# rfmodel = RandomForestRegressor(max_depth = 10)

# rfmodel.fit(X_train, y_train)

RandomForestRegressor(max_depth=10)

In [49]:
y_pred = cat_reg.predict(X_test)

In [ ]:
# y_pred = rfmodel.predict(X_test)

### 4. Sacar métricas, valorar los modelos 

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

In [50]:
print(np.sqrt(mean_squared_error(y_test, y_pred)))

322.27074296950826


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 1. Carga los datos de `test.csv` para predecir.


In [51]:
X_pred = pd.read_csv("data/test.csv", index_col = 0)
X_pred.index.name = None
X_pred.head()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
209,Lenovo,Legion Y520-15IKBN,Gaming,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD,Nvidia GeForce GTX 1060,No OS,2.4kg
1281,Acer,Aspire ES1-531,Notebook,15.6,1366x768,Intel Celeron Dual Core N3060 1.6GHz,4GB,500GB HDD,Intel HD Graphics 400,Linux,2.4kg
1168,Lenovo,V110-15ISK (i3-6006U/4GB/1TB/No,Notebook,15.6,1366x768,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,Intel HD Graphics 520,No OS,1.9kg
1231,Dell,Inspiron 7579,2 in 1 Convertible,15.6,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,2.191kg
1020,HP,ProBook 640,Notebook,14.0,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,4GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.95kg


In [52]:
X_pred.tail()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
820,MSI,GE72MVR 7RG,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,512GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,2.9kg
948,Toshiba,Tecra Z40-C-12X,Notebook,14.0,IPS Panel Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.47kg
483,Dell,Precision M5520,Workstation,15.6,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,256GB SSD,Nvidia Quadro M1200,Windows 10,1.78kg
1017,HP,Probook 440,Notebook,14.0,1366x768,Intel Core i5 7200U 2.5GHz,4GB,500GB HDD,Intel HD Graphics 620,Windows 10,1.64kg
421,Asus,ZenBook Flip,2 in 1 Convertible,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.27kg


In [53]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 209 to 421
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   Product           391 non-null    object 
 2   TypeName          391 non-null    object 
 3   Inches            391 non-null    float64
 4   ScreenResolution  391 non-null    object 
 5   Cpu               391 non-null    object 
 6   Ram               391 non-null    object 
 7   Memory            391 non-null    object 
 8   Gpu               391 non-null    object 
 9   OpSys             391 non-null    object 
 10  Weight            391 non-null    object 
dtypes: float64(1), object(10)
memory usage: 36.7+ KB


 ## 2. Replicar el procesado para ``test.csv``

In [54]:
X_pred["inches"] = X_pred.Inches.astype(float)
X_pred['ram_gb'] = X_pred['Ram'].str.replace('GB', '').astype(int)
X_pred["weight_kg"] = X_pred['Weight'].str.replace('kg', '').astype(float)
X_pred.drop(columns = ["Inches", "Ram", "Weight"], inplace = True)

In [55]:
X_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 209 to 421
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           391 non-null    object 
 1   Product           391 non-null    object 
 2   TypeName          391 non-null    object 
 3   ScreenResolution  391 non-null    object 
 4   Cpu               391 non-null    object 
 5   Memory            391 non-null    object 
 6   Gpu               391 non-null    object 
 7   OpSys             391 non-null    object 
 8   inches            391 non-null    float64
 9   ram_gb            391 non-null    int32  
 10  weight_kg         391 non-null    float64
dtypes: float64(2), int32(1), object(8)
memory usage: 35.1+ KB


In [56]:
# # Ordenamos las features categóricas de menor precio de median a mayor precio para poder convertirlas en numéricas:
# categories_company = df.groupby(df["Company"])["Price_in_euros"].median().sort_values().index.to_list()
# # categories_typename = df.groupby(df["TypeName"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_resolution = df.groupby(df["ScreenResolution"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_cpu = df.groupby(df["Cpu"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_memory = df.groupby(df["Memory"])["Price_in_euros"].median().sort_values().index.to_list()
# categories_gpu = df.groupby(df["Gpu"])["Price_in_euros"].median().sort_values().index.to_list()

# total_cat = [categories_company, categories_resolution, categories_cpu, categories_memory, categories_gpu]
# cols = ["Company", "ScreenResolution", "Cpu", "Memory", "Gpu"]

# enc = OrdinalEncoder(categories = total_cat, dtype= int)
# X_pred[cols] = enc.transform(X_pred[cols])

# Con la ram, el sistema operativo y el typename, vamos a hacer un get_dummies:
cols_dummies = X_train.columns
X_pred = pd.get_dummies(X_pred, columns = feat_sel, dtype = int).reindex(columns = cols_dummies, fill_value=0)

In [57]:
# X_pred.head()

In [59]:
predictions_submit = cat_reg.predict(X_pred)
predictions_submit

array([1425.79089012,  359.04998982,  357.94280744, 1079.94180303,
        841.74349492,  559.61790463,  842.87289997, 1059.39103287,
       1263.18331929,  459.22598446, 2137.2915774 , 1345.05459773,
        520.06154211, 1631.22220625, 1042.61548358,  619.16464806,
       3019.08158721, 1472.23201821, 2150.89787004,  657.91599661,
       1650.49603526,  509.95055068,  726.60589051, 1443.22411798,
        633.24088868,  729.18417396,  664.34543114,  948.3354448 ,
       2472.40134428, 1033.80301018, 2581.279052  ,  357.96579142,
        720.24860216, 3315.69077271, 2263.99028907, 1704.80911252,
        450.51465084, 1517.37096508,  914.80671993, 1867.69662662,
        793.54726202,  880.60579915,  697.79579273, 1448.25771416,
       1308.48811512, 1110.23134758,  980.97598789,  547.47502237,
        648.46118878,  544.68518652, 1981.02462515,  732.89528005,
       1046.81178069,  496.25190455, 1977.81852231, 1640.56993021,
        802.57658627,  838.36853072, 1173.88568887,  786.07036

### 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [60]:
sample = pd.read_csv("data/sample_submission.csv")

In [61]:
sample.head()

,laptop_ID,Price_in_euros
0,209,1949.1
1,1281,805.0
2,1168,1101.0
3,1231,1293.8
4,1020,1832.6


In [62]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [63]:
#¿Cómo creamos la submission?
submission = pd.DataFrame({"laptop_ID": X_pred.index,
                        "Price_in_euros": predictions_submit})

In [64]:
submission.head()

,laptop_ID,Price_in_euros
0,209,1425.790890
1,1281,359.049990
2,1168,357.942807
3,1231,1079.941803
4,1020,841.743495


In [65]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [66]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [67]:
chequeador(submission)

You're ready to submit!
